In [7]:
import pandas as pd
import yaml
import os
import plotly.express as px
import plotly
from plotly import graph_objects as go

In [8]:
max_dict = {
    'ar': 6074,
    'pl': 17721,
    'bg': 8906,
    'be': 22852,
    'fr': 14448,
    'en': 12542,
    'lv': 12520,
    'fa': 4797,
    'lt': 2340,
    'cs': 68494,
    'es': 14286, 
    'zh': 3996,
    'slk': 8482,
    'slv': 10902,
    'uk': 5495,
    'ru': 69629,
    'de': 13813,
    'fi': 12216,
    'id': 4481,
    'hr': 6913,
}

In [9]:
full_name = {
    "pl": "Polish",
    "ru": "Russian",
    "uk": "Ukrainian",
    "slv": "Slovene",
    "cs": "Czech",
    "be": "Belarussian",
    "de": "German",
    "slk": "Slovak",
    "fr": "French",
    "ar": "Arabic",
    "id": "Indonesian",
    "en": "English",
    "fi": "Finnish",
    "zh": "Chinese",
    "lv": "Latvian",
    "es": "Spanish"
}

In [10]:
slavic_langs = ("pl", "be", "cs", "slk", "slv", "uk", "ru")

In [11]:
RESULTS_CSV = '../probingOutputs/all_slavic/all_results.csv'

In [12]:
df = pd.read_csv(RESULTS_CSV)
print(df.columns)
print(df.dtypes)

Index(['train', 'test', 'model', 'layer', 'rank', 'limit', 'seed', 'directory',
       'dspear', 'uuas'],
      dtype='object')
train         object
test          object
model         object
layer          int64
rank           int64
limit        float64
seed           int64
directory     object
dspear       float64
uuas         float64
dtype: object


In [13]:
df['train'] = df.train.apply(lambda x: x[1:-1].replace("'", "").split(',')[0])
df['test'] = df.test.apply(lambda x: x[1:-1].replace("'", "").split(',')[0])
df["slavic"] = df.train.isin(slavic_langs)
df['limit'] = df.apply(lambda x: x.limit if not pd.isna(x.limit) else max_dict[x.train], axis=1)
df.head()

,train,test,model,layer,rank,limit,seed,directory,dspear,uuas,slavic
0,be,be,mBERT,6,128,100.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.416957,0.397976,True
1,be,be,mBERT,6,128,1000.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.711804,0.634634,True
2,be,be,mBERT,6,128,10000.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.779117,0.787562,True
3,be,be,mBERT,6,128,2500.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.756653,0.720526,True
4,be,be,mBERT,6,128,5000.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.770321,0.756249,True


In [14]:
avg_df = df.groupby(['train', 'test', 'model', 'layer', 'rank', 'limit']).mean().reset_index().drop(columns=["seed"])
avg_df["slavic"] = avg_df.slavic.astype(bool)

In [15]:
avg_df.head()

,train,test,model,layer,rank,limit,dspear,uuas,slavic
0,be,be,mBERT,6,128,100.0,0.424506,0.400528,True
1,be,be,mBERT,6,128,1000.0,0.714579,0.638396,True
2,be,be,mBERT,6,128,2500.0,0.752713,0.709843,True
3,be,be,mBERT,6,128,5000.0,0.770202,0.757244,True
4,be,be,mBERT,6,128,7500.0,0.775498,0.779258,True


In [16]:
for test in avg_df.test.unique():
    df_loc = avg_df[(avg_df.test == test) & (avg_df.limit <= 10000)]
    fig = px.scatter(data_frame=df_loc, x="limit", y="uuas", color="slavic", hover_data=["train"], labels={"limit": "# train sentences", "uuas": "UUAS", "slavic": "Is Slavic"}, title=f"{full_name[test]}")
    fig.update_layout(
        autosize=False,
        width=800,
        height=600,
        yaxis_range=[0, 1],
        font={"size":20}
    )
    fig.show()

In [17]:
for test in avg_df.test.unique():
    df_loc = avg_df[(avg_df.test == test) & (avg_df.limit <= 10000)]
    fig = px.line(data_frame=df_loc, x="limit", y="uuas", color="train", markers=True, title=full_name[test])
    fig.show()

In [18]:
df_loc = avg_df[(avg_df.limit <= 10000)].groupby(["train", "limit"]).mean().reset_index()
fig = px.line(data_frame=df_loc, x="limit", y="uuas", color="train", markers=True, title="Average")
fig.show()

In [19]:
sum(1 for x in max_dict if max_dict[x] >= 7500)

13

In [20]:
sum(1 for x in max_dict if max_dict[x] >= 10000) # wierszy?

11

In [21]:
len(slavic_langs) + 1 # kolumn

8

In [22]:
limit = 10000
print("test", end="")
for column in avg_df.test.unique():
    print(" & ", full_name[column], end="", sep="")
print(" & ", "Average", end="", sep="")
print(" \\\\")

selected_langs = (x for x in max_dict if max_dict[x] >= limit)
df_loc = avg_df[(avg_df.train.isin(selected_langs)) & (avg_df.limit == limit)].groupby("train").mean().uuas.reset_index().sort_values(by="uuas", ascending=False)
sorted_rows = df_loc.train.values

for row in sorted_rows:
    print(full_name[row], end="")
    for column in avg_df.test.unique():
        df_loc = avg_df[(avg_df.test == column) & (avg_df.train == row) & (avg_df.limit == limit)].uuas
        print(" & ", "%.2f" % (df_loc.values[0] * 100), end="", sep="")
    df_loc = avg_df[(avg_df.train == row) & (avg_df.limit == limit)].uuas.mean()
    print(" & ", "%.2f" % (df_loc * 100), end="", sep="")

    print(" \\\\")




test & Belarussian & Czech & Polish & Russian & Slovak & Slovene & Ukrainian & Average \\
Russian & 76.88 & 76.84 & 78.85 & 81.35 & 80.67 & 75.37 & 79.66 & 78.52 \\
Slovene & 75.26 & 77.05 & 78.58 & 76.19 & 82.36 & 81.53 & 77.16 & 78.30 \\
Polish & 76.03 & 77.06 & 83.20 & 76.74 & 81.21 & 75.68 & 78.04 & 78.28 \\
Czech & 74.86 & 80.48 & 78.21 & 76.32 & 83.20 & 76.08 & 77.48 & 78.09 \\
Belarussian & 78.70 & 74.02 & 76.44 & 75.55 & 78.11 & 72.88 & 77.31 & 76.14 \\
German & 73.13 & 74.24 & 76.15 & 74.58 & 78.01 & 73.24 & 74.98 & 74.90 \\
English & 70.40 & 71.38 & 73.79 & 73.14 & 75.90 & 70.16 & 73.82 & 72.65 \\
French & 70.96 & 70.96 & 75.11 & 72.30 & 74.44 & 70.13 & 72.95 & 72.41 \\
Latvian & 70.75 & 70.85 & 72.81 & 71.21 & 75.61 & 70.85 & 72.43 & 72.07 \\
Spanish & 69.76 & 70.22 & 74.05 & 71.06 & 72.41 & 69.72 & 71.86 & 71.30 \\
Finnish & 67.96 & 67.86 & 69.20 & 68.27 & 72.13 & 67.81 & 68.76 & 68.86 \\
